In [1]:
# Basic Imports
from omigo_core import tsv
import json
import urllib.parse

In [2]:
# Create simple TSV object having json strings as column values
def create_data1():
    # Create a simple tsv object with json data
    x = tsv.TSV("\t".join(["record", "json"]), [])
    
    # Mulitple hashmaps converted to json strings. As a best practice url encode textual data
    json_str1 = json.dumps({"name": "A1", "age": 20, "state": "CA"})
    json_str2 = json.dumps({"name": "A2", "age": 23, "state": "NY"})
    json_str3 = json.dumps({"name": "A3", "age": 42, "state": "WA"})

    # Add the json strings as rows
    return x \
        .add_map_as_row({"record": "1", "json": urllib.parse.quote_plus(json_str1)}) \
        .add_map_as_row({"record": "2", "json": urllib.parse.quote_plus(json_str2)}) \
        .add_map_as_row({"record": "3", "json": urllib.parse.quote_plus(json_str3)})

In [3]:
# read sample data
x1 = create_data1()
x1.to_df()

,record,json
0,1,%7B%22name%22%3A+%22A1%22%2C+%22age%22%3A+20%2...
1,2,%7B%22name%22%3A+%22A2%22%2C+%22age%22%3A+23%2...
2,3,%7B%22name%22%3A+%22A3%22%2C+%22age%22%3A+42%2...


In [4]:
# use explode_json() api to convert the json strings to tabular structure.
# Notice the special variables __json_index__. When the json element is an array,
# the added index reflects which output rows came from the same json
y1 = x1.explode_json("json", prefix = "parsed")
y1.to_df()

,parsed:__json_index__,record,parsed:age,parsed:name,parsed:state
0,1,1,20,A1,CA
1,2,2,23,A2,NY
2,3,3,42,A3,WA


In [5]:
# Create more complex TSV object having multiple level json strings as column values
def create_data2():
    # Create a simple tsv object with json data
    x = tsv.TSV("\t".join(["record", "json"]), [])
    
    # Mulitple hashmaps converted to json strings. As a best practice url encode textual data
    json_str1 = json.dumps({"name": "A1", "age": 20, "state": "CA", "education": [{"degree": "BS", "year": "2000"}]})
    json_str2 = json.dumps({"name": "A2", "age": 23, "state": "NY", "education": [{"degree": "BS", "year": "2004"}, {"degree": "MS", "year": "2006"}]})
    json_str3 = json.dumps({"name": "A3", "age": 42, "state": "WA", "education": [{"degree": "MS", "year": "2010"}, {"degree": "PHD", "year": "2015"}]})

    # Add the json strings as rows
    return x \
        .add_map_as_row({"record": "1", "json": urllib.parse.quote_plus(json_str1)}) \
        .add_map_as_row({"record": "2", "json": urllib.parse.quote_plus(json_str2)}) \
        .add_map_as_row({"record": "3", "json": urllib.parse.quote_plus(json_str3)})

In [6]:
# read sample data
x2 = create_data2()
x2.to_df()

,record,json
0,1,%7B%22name%22%3A+%22A1%22%2C+%22age%22%3A+20%2...
1,2,%7B%22name%22%3A+%22A2%22%2C+%22age%22%3A+23%2...
2,3,%7B%22name%22%3A+%22A3%22%2C+%22age%22%3A+42%2...


In [7]:
# use explode_json() api to convert the json strings to tabular structure.
# Notice the special variables __json_index__. When the json element is an array,
# the added index reflects which output rows came from the same json
y2 = x2 \
    .explode_json("json", prefix = "parsed")
y2.to_df()

,parsed:__json_index__,record,parsed:age,parsed:education:__explode_json_len__,parsed:education:degree,parsed:education:year,parsed:name,parsed:state
0,1,1,20,1,BS,2000,A1,CA
1,2,2,23,2,BS,2004,A2,NY
2,2,2,23,2,MS,2006,A2,NY
3,3,3,42,2,MS,2010,A3,WA
4,3,3,42,2,PHD,2015,A3,WA
